In [1]:
import pandas as pd
import os
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
def load_data(file_path):
    df = pd.read_csv(file_path)
    print(f'Data successfully loaded with {df.shape[0]} rows and {df.shape[1]} features')
    return df

train_set = load_data('../data/processed/train_processed_v1.csv')
dev_set = load_data('../data/processed/dev_processed_v1.csv')
test_set = load_data('../data/processed/test_processed_v1.csv')

Data successfully loaded with 4751 rows and 19 features
Data successfully loaded with 1356 rows and 19 features
Data successfully loaded with 910 rows and 19 features


In [3]:
def split_feature_target(df, target_variable: str = 'selling_price_log'):
    y = df[target_variable]
    x = df.drop(columns=[target_variable])
    print(f'target_shape: {y.shape} - feature_shape: {x.shape}')
    return x, y

x_train, y_train = split_feature_target(train_set)
x_dev, y_dev = split_feature_target(dev_set)
x_test, y_test = split_feature_target(test_set)

target_shape: (4751,) - feature_shape: (4751, 18)
target_shape: (1356,) - feature_shape: (1356, 18)
target_shape: (910,) - feature_shape: (910, 18)


In [4]:
# import libraries
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
import optuna
from sklearn.metrics import root_mean_squared_error, r2_score
import time

In [5]:
def format_duration(seconds: float) -> str:
    if seconds < 60:
        return f"{seconds:.2f}s"
    elif seconds < 3600:
        minutes = int(seconds // 60)
        secs = seconds % 60
        return f"{minutes}m {secs:.2f}s"
    else:
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        secs = seconds % 60
        return f"{hours}h {minutes}m {secs:.2f}s"

In [6]:
# baseline model training
baseline_models = {
    'Ridge' : Ridge(alpha=0.3, max_iter=2000, random_state=1),
    'XGBoost' : XGBRegressor(random_state=42, n_estimators=200, max_depth=3),
    'LightGBM' : LGBMRegressor(max_depth=5, learning_rate=0.3, n_estimators=200, random_state=42),
    'RandomForest' : RandomForestRegressor(n_estimators=200, max_depth=4, random_state=42)
}

In [21]:
results = {}
print("="*50)
print(f"BASELINE TRAINING INITIALIZED")
print("="*50)
for name, model in baseline_models.items():
    print(f'Training {name} model...')
    start_time = time.time()
    model.fit(x_train, y_train)
    time_elapsed = time.time() - start_time

    preds_train = model.predict(x_train)

    results[name] = {
        "Time_elapsed" : format_duration(time_elapsed),
        "R^2_score" : round(r2_score(y_train, preds_train),4),
        "RMSE" : round(root_mean_squared_error(y_train, preds_train), 4)
    }

for name, result in results.items():
        print(f"{name}")
        print('-'*20)
        print("Time elapsed: ",result['Time_elapsed'])
        print("R^2_Score: ", result['R^2_score'])
        print("RMSE: ",result['RMSE'])
        print ("="*30)

print(f'BASELINE MODEL TRAINING COMPLETED')

BASELINE TRAINING INITIALIZED
Training Ridge model...
Training XGBoost model...
Training LightGBM model...
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000315 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 870
[LightGBM] [Info] Number of data points in the train set: 4751, number of used features: 18
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

In [8]:
results = {}
print("="*50)
print(f"CROSS VALIDATION INITIALIZED")
print(f"="*50)

for name, model in baseline_models.items():
    start_time = time.time()
    cv_scores = cross_val_score(model, x_train, y_train, cv=KFold(5), scoring='neg_root_mean_squared_error', n_jobs=-1)
    time_elapsed = time.time() - start_time

    abs_cv_score = abs(cv_scores)
    
    results[name] = {
        'cv_scores_across_folds' : abs_cv_score,
        'cv_score_mean' : round(abs_cv_score.mean(), 4),
        'cv_score_std' : round(abs_cv_score.std(), 4),
        'Time_elapsed' : format_duration(time_elapsed)
    }

for name, result in results.items():
    print(f'Model_name: {name}')
    print(f'='*40)
    print(f'CV Scores: ',result['cv_scores_across_folds'])
    print(f'CV scores_mean: ',result['cv_score_mean'])
    print(f'CV score_std: +/-',result['cv_score_std'])
    print(f'Time_elapsed: ',result['Time_elapsed'])
    print(f'='*40)

print(f'CROSS-VALIDATION MODEL TRAINING COMPLETED')

CROSS VALIDATION INITIALIZED
Model_name: Ridge
CV Scores:  [0.41716765 0.3952506  0.39828129 0.39223018 0.40086055]
CV scores_mean:  0.4008
CV score_std: +/- 0.0087
Time_elapsed:  6.56s
Model_name: XGBoost
CV Scores:  [0.29129551 0.30042265 0.2986982  0.27874239 0.29366269]
CV scores_mean:  0.2926
CV score_std: +/- 0.0077
Time_elapsed:  4.62s
Model_name: LightGBM
CV Scores:  [0.28973033 0.30676242 0.3128978  0.29391002 0.30420172]
CV scores_mean:  0.3015
CV score_std: +/- 0.0085
Time_elapsed:  3.87s
Model_name: RandomForest
CV Scores:  [0.44352197 0.41886004 0.40985105 0.42159388 0.41798678]
CV scores_mean:  0.4224
CV score_std: +/- 0.0113
Time_elapsed:  1.57s
CROSS-VALIDATION MODEL TRAINING COMPLETED


In [9]:
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner

In [ ]:
def objective(trial):
            n_estimators = trial.suggest_int('n_estimators', 100,1000)
            max_depth = trial.suggest_int('max_depth', 3, 15)
            learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.3)
            min_child_weight = trial.suggest_int('min_child_weight', 1 , 10)

            xgbr = XGBRegressor(
                n_estimators=n_estimators,
                max_depth=max_depth,
                learning_rate=learning_rate,
                min_child_weight=min_child_weight,
                random_state=42
            )


            scores = cross_val_score(
                    estimator=xgbr,
                    X=x_train, y=y_train,
                    cv=KFold(5),scoring='neg_mean_squared_error',
                    n_jobs=-1,
            )

            return -scores.mean()

In [ ]:
study = optuna.create_study(direction='minimize',sampler=TPESampler(seed=42),pruner=MedianPruner())
study.optimize(objective, n_trials=160,show_progress_bar=False)

[I 2026-01-21 16:07:13,991] A new study created in memory with name: no-name-2b535705-1135-4482-8846-3e83ab50ba39
[I 2026-01-21 16:07:20,520] Trial 0 finished with value: 0.08539019019252586 and parameters: {'n_estimators': 425, 'max_depth': 4, 'learning_rate': 0.08978658960089328, 'min_child_weight': 8}. Best is trial 0 with value: 0.08539019019252586.
[I 2026-01-21 16:07:26,413] Trial 1 finished with value: 0.09495197683593737 and parameters: {'n_estimators': 662, 'max_depth': 12, 'learning_rate': 0.04225670585219596, 'min_child_weight': 9}. Best is trial 0 with value: 0.08539019019252586.
[I 2026-01-21 16:07:26,957] Trial 2 finished with value: 0.08561203246811563 and parameters: {'n_estimators': 390, 'max_depth': 6, 'learning_rate': 0.09152415967258339, 'min_child_weight': 10}. Best is trial 0 with value: 0.08539019019252586.
[I 2026-01-21 16:07:29,506] Trial 3 finished with value: 0.10226171316013058 and parameters: {'n_estimators': 911, 'max_depth': 12, 'learning_rate': 0.0849960

In [24]:
import matplotlib.pyplot as plt
import plotly
optuna.visualization.plot_optimization_history(study)

In [25]:
optuna.visualization.plot_param_importances(study)

In [26]:
optuna.visualization.plot_slice(study)

In [27]:
best_params = study.best_params

In [28]:
best_n_estimators = best_params['n_estimators']
best_max_depth = best_params['max_depth']
best_learning_rate = best_params['learning_rate']
best_min_child_weight = best_params['min_child_weight']

In [29]:
best_model = XGBRegressor(
    n_estimators=best_n_estimators,
    max_depth=best_max_depth,
    learning_rate=best_learning_rate,
    min_child_weight=best_min_child_weight,
    random_state=42
)

In [30]:
best_model.fit(x_train, y_train)

,"objective objective: typing.Union[str, xgboost.sklearn._SklObjWProto, typing.Callable[[typing.Any, typing.Any], typing.Tuple[numpy.ndarray, numpy.ndarray]], NoneType]Specify the learning task and the corresponding learning objective or a customobjective function to be used.For custom objective, see :doc:`/tutorials/custom_metric_obj` and:ref:`custom-obj-metric` for more information, along with the end note forfunction signatures.",'reg:squarederror'
,"base_score base_score: typing.Union[float, typing.List[float], NoneType]The initial prediction score of all instances, global bias.",None
,booster,None
,"callbacks callbacks: typing.Optional[typing.List[xgboost.callback.TrainingCallback]]List of callback functions that are applied at end of each iteration.It is possible to use predefined callbacks by using:ref:`Callback API `... note:: States in callback are not preserved during training, which means callback objects can not be reused for multiple training sessions without reinitialization or deepcopy... code-block:: python for params in parameters_grid: # be sure to (re)initialize the callbacks before each run callbacks = [xgb.callback.LearningRateScheduler(custom_rates)] reg = xgboost.XGBRegressor(**params, callbacks=callbacks) reg.fit(X, y)",None
,colsample_bylevel colsample_bylevel: typing.Optional[float]Subsample ratio of columns for each level.,None
,colsample_bynode colsample_bynode: typing.Optional[float]Subsample ratio of columns for each split.,None
,colsample_bytree colsample_bytree: typing.Optional[float]Subsample ratio of columns when constructing each tree.,None
,"device device: typing.Optional[str].. versionadded:: 2.0.0Device ordinal, available options are `cpu`, `cuda`, and `gpu`.",None
,"early_stopping_rounds early_stopping_rounds: typing.Optional[int].. versionadded:: 1.6.0- Activates early stopping. Validation metric needs to improve at least once in every **early_stopping_rounds** round(s) to continue training. Requires at least one item in **eval_set** in :py:meth:`fit`.- If early stopping occurs, the model will have two additional attributes: :py:attr:`best_score` and :py:attr:`best_iteration`. These are used by the :py:meth:`predict` and :py:meth:`apply` methods to determine the optimal number of trees during inference. If users want to access the full model (including trees built after early stopping), they can specify the `iteration_range` in these inference methods. In addition, other utilities like model plotting can also use the entire model.- If you prefer to discard the trees after `best_iteration`, consider using the callback function :py:class:`xgboost.callback.EarlyStopping`.- If there's more than one item in **eval_set**, the last entry will be used for early stopping. If there's more than one metric in **eval_metric**, the last metric will be used for early stopping.",None
,enable_categorical enable_categorical: boolSee the same parameter of :py:class:`DMatrix` for details.,False
,"eval_metric eval_metric: typing.Union[str, typing.List[typing.Union[str, typing.Callable]], typing.Callable, NoneType].. versionadded:: 1.6.0Metric used for monitoring the training result and early stopping. It can be astring or list of strings as names of predefined metric in XGBoost (See:doc:`/parameter`), one of the metrics in :py:mod:`sklearn.metrics`, or anyother user defined metric that looks like `sklearn.metrics`.If custom objective is also provided, then custom metric should implement thecorresponding reverse link function.Unlike the `scoring` parameter commonly used in scikit-learn, when a callableobject is provided, it's assumed to be a cost function and by default XGBoostwill minimize the result during early stopping.For advanced usage on Early stopping like directly choosing to maximize insteadof minimize, see :py:obj:`xgboost.callback.EarlyStopping`.See :doc:`/tutorials/custom_metric_obj` and :ref:`custom-obj-metric` for moreinformation... code-block:: python from sklearn.datasets import load_diabetes

In [31]:
y_pred = best_model.predict(x_test)

In [32]:
from sklearn.metrics import root_mean_squared_error
rmse = root_mean_squared_error(y_test, y_pred)
rmse

0.2609738229239942

In [33]:
y_pred = best_model.predict(x_train)

In [34]:
rmse = root_mean_squared_error(y_train, y_pred)

In [ ]:
rmse